<a href="https://colab.research.google.com/github/vijayamurugan27/GenerativeAI/blob/main/Advanced_Agents_use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q langchain_groq langchain_community gradio langgraph nasapy polygon arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [2]:
import os

from langchain_groq import ChatGroq
from google.colab import userdata
llm_groq = ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview", api_key=userdata.get("GROQ_API_KEY"))
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_SEARCH")
os.environ["POLYGON_API_KEY"] = userdata.get("POLYGON_API_KEY")

In [3]:
from datetime import datetime
import json
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [4]:
from langchain_core.tools import tool # For custom tools
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv import ArxivQueryRun
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.tools.polygon import PolygonFinancials, PolygonAggregates, PolygonTickerNews, PolygonLastQuote
from langchain_community.tools.youtube.search import YouTubeSearchTool
from nasapy import Nasa
import polygon

In [5]:
"""
We will first setup the tools needed to run by the agent. We will use search engine API call
and one custom function that will at this point do nothing.

You can find standard list of tools here: https://python.langchain.com/v0.2/docs/integrations/tools/
A lot of them are noise, but some of them are useful.
"""
search = TavilySearchResults(max_results=2)
paper = ArxivQueryRun()
yt = YouTubeSearchTool()
stocks_client = polygon.StocksClient(os.getenv("POLYGON_API_KEY"))
stock_financials = PolygonFinancials(api_wrapper = PolygonAPIWrapper())
stock_ticker_news = PolygonTickerNews(api_wrapper = PolygonAPIWrapper())

@tool
def get_hotel(location: str, at_time: datetime)->str:
    '''Returns hotel options for reservation'''
    return '''There is a beautiful Motel 6 available for that date'''

@tool
def get_nasa_pic_of_day()->str:
    """Get NASA's Astronomy Picture of the Day today."""
    date = datetime.now().strftime("%Y-%m-%d")
    nasa = Nasa()
    apod = nasa.picture_of_the_day(date, hd=True)
    return apod['url']

@tool
def get_stock_price(ticker: str)->str:
    '''Returns the latest stock price'''
    return stocks_client.get_previous_close(ticker)

In [6]:
tools = [search, paper, stock_financials, yt, stock_ticker_news, get_hotel, get_nasa_pic_of_day, get_stock_price]
prompt = """We will be using tools to solve a variety of user problems. This will be displayed on Gradio.
              The output has to be in markdown format to display properly. For weather and search you will use Tavily"""
memory = MemorySaver()
agent_executor = create_react_agent(llm_groq, tools, checkpointer=memory, state_modifier=prompt)
config = {"configurable": {"thread_id": "id111"}} # Hardcoded -- this is actually to switch different threads of conversations

In [7]:
# Setup the function to handle chat messages from the user
def chat(message, history):
    tokens_used = 0
    tool_name = None
    query = None
    for chunk in agent_executor.stream(
        {"messages": [HumanMessage(content=message)]}, config):

        #print(chunk)
        agent_data = chunk.get("agent", None)

        if agent_data is not None:
            messages = agent_data.get("messages", None)

            if messages is not None and len(messages) > 0:
                message = messages[0]  # Corrected to refer to the first message
                tool_calls = message.additional_kwargs.get("tool_calls", None)

                if tool_calls is not None and len(tool_calls) > 0:
                    tool_name = tool_calls[0].get("function", {}).get("name", None)
                    arguments_str = tool_calls[0].get("function", {}).get("arguments", '{}')
                    #print(tool_calls)
                    arguments = json.loads(arguments_str)  # Parse the JSON string into a dictionary
                    query = arguments.get("query", None)  # Now safely access the query

                tokens_used = message.response_metadata['token_usage']['total_tokens']
                if tool_name is not None and query is not None:
                    yield message.content + "\nTokens used so far: " + str(tokens_used)+"\nTool used: "+tool_name+"\nQuery: "+query

                elif tool_name is not None:
                    yield message.content + "\nTokens used so far: " + str(tokens_used)+"\nTool used: "+tool_name

                else:
                    yield message.content

In [8]:
import gradio as gr
demo = gr.ChatInterface(
    fn=chat,
    title="Advanced chat agent",
)
demo.launch()

# can you do hotel booking?
# Get nasa's picture of the day
# Nutrition info or home made roti
# how many Infosys stocks can i buy for $1000?
# financials for AAPL
# compare that against NVDA
# which stock is better in comparison
# get me latest news for MSFT ticker

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:231: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5ea88aea8be32bd8f1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
